# Курсовая работа № 2
## Аналитика. Начальный уровень

Курсовая работа состоит из двух частей – обязательной и дополнительной. **Для зачета необходимо выполнение только первой части.** Выполнение второй части может потребовать дополнительные знания Python.

- [Часть первая](#Часть-первая)
- [Часть вторая](#Часть-вторая)

## Часть первая


Перед вами стоит задача – подготовить аналитический отчет для HR-отдела. На основании проведенной аналитики предполагается составить рекомендации для отдела кадров по стратегии набора персонала, а также по взаимодействию с уже имеющимися сотрудниками.
<br><br> В базе данных лежит набор таблиц, которые содержат данные о сотрудниках вымышленной компании.
Сделайте обзор штата сотрудников компании. Составьте набор предметов исследования, а затем проверьте их на данных. Вся аналитика должна быть выполена с помощью SQL. Впоследствии данные можно визуализировать, однако финальные датафреймы для графиков также должны быть подготовлены с помощью SQL. <br><br>

Примеры гипотез:

1. Есть зависимость между `perfomance score` и тем, под чьим руководством работает сотрудник.
2. Есть зависимость между продолжительностью работы в компании и семейным положением сотрудника.
2. Есть зависимость между продолжительностью работы в компании и возрастом сотрудника.

<br><br>
Параметры для подключения следующие: 

хост – `dsstudents.skillbox.ru`,<br> порт – `5432`,<br> имя базы данных – `human_resources`,<br> пользователь – `readonly`,<br> пароль – `6hajV34RTQfmxhS`.<br><br> 
Таблицы, доступные для анализа, – `hr_dataset`, `production_staff`, `recruiting_costs`, `salary_grid`.

In [414]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "human_resources",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)

In [415]:
import pandas as pd

sql = lambda q: pd.read_sql_query(q, conn)

def sqlexec(q: str, commit=True):
    with conn.cursor() as c:
        return c.execute(q)
    if commit:
        conn.commit()

### Посмотрим на данные

In [416]:
# Managers
sql(
    """
    select "Manager Name" mn, count("Manager Name") c
    from hr_dataset 
    group by mn
    order by c, mn;
    """
)

,mn,c
0,Board of Directors,2
1,Debra Houlihan,3
2,Eric Dougall,4
3,Brandon R. LeBlanc,7
4,Jennifer Zamora,7
5,Brian Champaigne,8
6,Alex Sweetwater,9
7,Lynn Daneault,13
8,John Smith,14
9,Peter Monroe,14


In [417]:
# Departments
sql(
    """
    select department d, count(department) c 
    from hr_dataset 
    group by d
    order by c, d;
    """
)

,d,c
0,Executive Office,1
1,Admin Offices,10
2,Software Engineering,10
3,Sales,31
4,IT/IS,50
5,Production,208


In [418]:
# Races
sql(
    """
    select racedesc r, count(racedesc) c 
    from hr_dataset 
    group by r
    order by c, r;
    """
)

,r,c
0,American Indian or Alaska Native,4
1,Hispanic,4
2,Two or more races,18
3,Asian,34
4,Black or African American,57
5,White,193


In [419]:
# Term reasons
sql(
    """
    select "Reason For Term" tr, count("Reason For Term") c 
    from hr_dataset 
    group by tr 
    order by c, tr;
    """
)

,tr,c
0,gross misconduct,1
1,maternity leave - did not return,3
2,medical issues,3
3,"no-call, no-show",3
4,military,4
5,performance,4
6,retiring,4
7,relocation out of area,5
8,return to school,5
9,attendance,7


In [420]:
# Employee sources
sql(
    """
    select "Employee Source" es, count(*) c
    from hr_dataset hd 
    group by es
    order by c desc, es;
    """
)

,es,c
0,Employee Referral,31
1,Diversity Job Fair,29
2,Search Engine - Google Bing Yahoo,25
3,Monster.com,24
4,Pay Per Click - Google,21
5,Professional Society,20
6,Newspager/Magazine,18
7,MBTA ads,17
8,Billboard,16
9,Vendor Referral,15


In [421]:
# Check if production_staff and hr_dataset have some commont employees.
# Also check that hd."Pay Rate" and ps."Pay" have the same meaning.
#
# Looks like the answer is yes to both questions.
sql(
    """
    select distinct hd."Employee Name", "Pay Rate", "Pay" 
    from            hr_dataset hd 
        inner join production_staff ps 
        on hd."Employee Name" = ps."Employee Name";
    """     
)

,Employee Name,Pay Rate,Pay
0,"Adinolfi, Wilson K",20.00,$20.00
1,"Akinkuolie, Sarah",29.00,$29.00
2,"Alagbe,Trina",21.00,$21.00
3,"Albert, Michael",54.50,$54.50
4,"Anderson, Carol",16.00,$16.00
...,...,...,...
203,"Winthrop, Jordan",29.00,$29.00
204,"Wolk, Hang T",22.00,$22.00
205,"Woodson, Jason",24.25,$24.25
206,"Ybarra, Catherine",22.00,$22.00


In [422]:
# Performances
# PIP stands for performance improvement plan.
sql(
    """    
    select "Performance Score" ps, count(*) c
    from hr_dataset hd 
    group by ps
    order by c, ps;
    """
)

,ps,c
0,Exceptional,9
1,PIP,9
2,Needs Improvement,15
3,Exceeds,28
4,90-day meets,31
5,N/A- too early to review,37
6,Fully Meets,181


### Подготовим данные для более детального анализа

In [423]:
# Let's orginize ages into bins splitead at each 5 years interval
sqlexec(
"""
create or replace function age_group(age int) 
returns int as $$
begin
    return (age :: int / 5) * 5;
end;
$$ language plpgsql;
"""
)

sql(
    """
    select age_group(age) as bin,
        count(1) c
    from hr_dataset hd
    group by bin
    order by bin;
    """
)

,bin,c
0,25,36
1,30,88
2,35,64
3,40,46
4,45,38
5,50,23
6,55,5
7,60,4
8,65,6


Нужно преобразовать качественную оценку произовдительности в количествунную, чтобы можно было что-то осмысленное вычислить по этим данным. Выбор конкретного преобразования бедет влиять на различные метрики, вычисляемые по полученным значениям, однако некоторорый "разумный" выбор семенства преобразований будет обладать тем свойством, что метрики, посчитанные по разным представителям семества, не будут отличаться друг слишком сильно (аутсайдеры будут оставаться аутсайдерами, лидеры — лидерами, но могут быть перестановки внутри каждой группы). 

In [424]:
sqlexec(
    """
    create or replace function perf_score(score_name Text)
    returns int as $$
        select score from ( 
            values 
            ('Exceptional', 2),
            ('Exceeds', 1),
            ('Fully Meets', 1),
            ('90-day meets', 1),
            ('N/A- too early to review', 0),
            ('Needs Improvement', -1),
            ('PIP', -2)
        ) as t (code_name, score)
        where score_name = code_name;
    $$ language sql;
    """
)

Для удобства анализа продолжительности работы в компании, разобьем продолжительность на несколько категорий.

In [425]:
sqlexec(
    """
    create or replace function days_to_period(days int)
    returns int as $$
    begin 
        return width_bucket(days, array[90, 180, 360, 640, 720, 1400]);
    end
    $$ language plpgsql;
    """
)

sqlexec(
    """
    create or replace function days_to_period_name(days int)
    returns text as $$
    declare 
        bin int;
    begin 
        bin := width_bucket(days, array[90, 180, 360, 640, 720, 1400]);
        return (array[
            'less then tree months', 
            'tree months', 
            'half a year', 
            'one year', 
            'a year and a half', 
            'two years', 
            'more then two years'
        ])[bin + 1];
    end
    $$ language plpgsql;
    """
)

Избавимся от редких значений, мешающих аналлизу.


In [426]:
# Managers
sqlexec(
    """create temp table if not exists Manager as
    select "Manager Name" as name, count(*) c
    from hr_dataset hd
    group by name
    -- 10 would be better, but we have what we have.
    having count(*) >= 7 
    order by c;
    """
)
assert not sql("select count(*) from Manager").empty

# Departments
sqlexec(
    """create temp table if not exists Department as
    select department dep, count(*) c
    from hr_dataset hd
    group by dep
    having count(*) >= 10 
    order by c;
    """
)
assert not sql("select count(*) from Department").empty


# Employee sources
sqlexec(
    """create temp table if not exists EmployeeSource as
    select "Employee Source" as source, count(*) c
    from hr_dataset hd
    group by source
    having count(*) >= 10 
    order by c;
    """
)
assert not sql("select count(*) from EmployeeSource").empty


# Race descriptions
sqlexec(
    """create temp table if not exists Races as
    select racedesc race, count(*) c
    from hr_dataset hd
    group by race
    having count(*) >= 10 
    order by c;
    """
)
assert not sql("select count(*) from Races").empty


In [427]:
# Let's create some views that would help with filtering.
sqlexec(
    """
    create or replace temp view managersv as (
        select * from hr_dataset hd 
        where hd."Manager Name" in (select name from Manager)
    );
    """
)
assert not sql("select count(*) from managersv").empty

sqlexec(
    """
    create or replace temp view departmentsv as (
        select * from hr_dataset hd 
        where hd.department in (select dep from Department)
    );
    """
)
assert not sql("select count(*) from departmentsv").empty

sqlexec(
    """
    create or replace temp view racesv as (
        select * from hr_dataset hd 
        where hd.racedesc in (select race from Races)
    );
    """
)
assert not sql("select count(*) from racesv").empty

sqlexec(
    """
    create or replace temp view employeesourcesv as (
        select * from hr_dataset hd
        where "Employee Source" in (select source from EmployeeSource)
    );
    """
)
assert not sql("select count(*) from employeesourcesv").empty

### Приступим к анализу
#### Посмотрим, правда ли, что работать под руководством некоторых менеджеров финансово выгоднее, чем под руководством других.

In [428]:
sql(
    """
    select "Manager Name" as name, 
           round(avg("Pay Rate") :: numeric, 2) mean, 
           round(min("Pay Rate") :: numeric, 2) min,
           round(max("Pay Rate") :: numeric, 2) max,
           count("Pay Rate")
    from managersv
    group by name
    order by mean desc;
    """
)

,name,mean,min,max,count
0,Lynn Daneault,55.31,55.00,57.00,13
1,John Smith,55.14,54.00,56.00,14
2,Jennifer Zamora,52.21,21.00,65.00,7
3,Janet King,52.13,33.50,65.00,19
4,Alex Sweetwater,51.07,45.42,57.12,9
5,Brian Champaigne,48.69,45.00,55.00,8
6,Peter Monroe,44.96,27.00,56.20,14
7,Simon Roup,44.31,30.20,62.00,17
8,Brandon R. LeBlanc,24.86,16.56,34.95,7
9,Brannon Miller,21.70,15.00,29.00,21


Да, это правда, но интереснее и чеснее посмотреть на это в разбивке по департаментам.

In [429]:
sql(
    """
    select "Manager Name" as name, 
           department dep, 
           round(avg("Pay Rate") :: numeric, 2) mean, 
           round(min("Pay Rate") :: numeric, 2) min,
           round(max("Pay Rate") :: numeric, 2) max,
           count("Pay Rate")
    from managersv mv
    where mv.department in (select d.dep from Department d)
    group by name, dep
    order by dep, mean desc;
    """
)

,name,dep,mean,min,max,count
0,Janet King,Admin Offices,55.00,55.00,55.00,2
1,Brandon R. LeBlanc,Admin Offices,24.86,16.56,34.95,7
2,Janet King,IT/IS,65.00,65.00,65.00,1
3,Jennifer Zamora,IT/IS,56.42,21.00,65.00,6
4,Brian Champaigne,IT/IS,48.69,45.00,55.00,8
5,Peter Monroe,IT/IS,44.96,27.00,56.20,14
6,Simon Roup,IT/IS,44.31,30.20,62.00,17
7,Janet King,Production,50.37,33.50,60.00,15
8,Brannon Miller,Production,21.70,15.00,29.00,21
9,Amy Dunn,Production,21.29,15.00,29.00,21


Внутри департамента разница не так замента, но все же присутствует. Также можно предположить, что менеджеры с небольшим количеством подчиненных и большим матожиданием по зп, занимают управляют всем департаментом.

#### Что насчет эфферктивности работы команды под руководством определенного менеджера, есть ли заметная разница?

In [430]:
sql(
    """
    select "Manager Name" as name, 
           round(avg(perf_score("Performance Score")) :: numeric, 2) mean,
           round(stddev(perf_score("Performance Score")) :: numeric, 2) std, 
           min(perf_score("Performance Score")) min,
           max(perf_score("Performance Score")) max,
           count("Performance Score")
    from managersv
    group by name
    order by mean desc;
    """
)

,name,mean,std,min,max,count
0,Brian Champaigne,1.00,0.00,1,1,8
1,Kelley Spirea,0.95,0.21,0,1,22
2,Alex Sweetwater,0.89,0.78,-1,2,9
3,Simon Roup,0.88,0.60,0,2,17
4,Jennifer Zamora,0.86,0.90,-1,2,7
5,Janet King,0.84,0.69,-1,2,19
6,Ketsia Liebig,0.81,0.51,-1,1,21
7,David Stanley,0.81,0.40,0,1,21
8,Kissy Sullivan,0.77,0.81,-2,2,22
9,Elijiah Gray,0.77,0.53,-1,1,22


Разница в работе команд под управлением разных менеджеров очень велика между лидерами и аутсайдерами. 

Посмотрим, связан ли этот факт только с управленческими качествами менеджера или же департаменд значимую роль в производительности команды. 

In [431]:
sql(
    """
    select "Manager Name" as name,
           department dep,
           round(avg(perf_score("Performance Score")) :: numeric, 2) mean,
           round(stddev(perf_score("Performance Score")) :: numeric, 2) std, 
           min(perf_score("Performance Score")) min,
           max(perf_score("Performance Score")) max,
           count("Performance Score")
    from managersv mv
    where mv.department in (select d.dep from Department d)
    group by dep, name
    having count("Performance Score") >= 10
    order by dep, mean desc;
    """
)

,name,dep,mean,std,min,max,count
0,Simon Roup,IT/IS,0.88,0.60,0,2,17
1,Peter Monroe,IT/IS,0.64,0.50,0,1,14
2,Kelley Spirea,Production,0.95,0.21,0,1,22
3,Ketsia Liebig,Production,0.81,0.51,-1,1,21
4,David Stanley,Production,0.81,0.40,0,1,21
5,Elijiah Gray,Production,0.77,0.53,-1,1,22
6,Kissy Sullivan,Production,0.77,0.81,-2,2,22
7,Janet King,Production,0.73,0.70,-1,1,15
8,Amy Dunn,Production,0.67,0.80,-2,2,21
9,Webster Butler,Production,0.67,0.66,-1,1,21


Кажется, что производительность команды сильно связана с тем, кто ей управляет. Однако, тут нужно сделать дополнительное исследование, которое прольет свет на то, отличаются ли задачи команд лидеров от команд аутсайдеров из одного отдела; на то, как были сформированы эти команды; и т.д.

#### Посмотрим, есть ли разница в производительности между отдделами.



In [432]:
sql(
    """
    select department dep,
        round(avg(perf_score("Performance Score")) :: numeric, 2) mean,
        round(stddev(perf_score("Performance Score")) :: numeric, 2) std, 
        min(perf_score("Performance Score")) min,
        max(perf_score("Performance Score")) max,
        count("Performance Score")
    from departmentsv
    group by dep
    order by mean desc;
    """
)

,dep,mean,std,min,max,count
0,Software Engineering,0.90,0.74,-1,2,10
1,IT/IS,0.86,0.57,-1,2,50
2,Admin Offices,0.80,0.42,0,1,10
3,Production,0.69,0.75,-2,2,208
4,Sales,0.65,0.88,-2,1,31


Выходит, что отделы, связанный с информационными технологиями показываюn лучшую производительность, чем продажи и производство. 

Это может быть связано с, но не огрничиваться, следующими возможностями:

1. Неадекватно высокими целевыми KPI для оделов аутсайдеров;
1. Низкими целевыми KPI для отделов лидеров;
1. Отсуствием налаженных процессов и наработанных методологий в одтелах аутсайдерах;
1. Неудачным периодом исследования, например кризис на рынке, из-за чего продажники плохо продают, а  производство срывает сроки из-за поставщиков...

##### Каснемся несколько деликатной темы: есть ли разница в производительности представителй рвзных расс в конкретной компании.

In [433]:
sql(
    """
    select racedesc race, 
           round(avg(perf_score("Performance Score")) :: numeric, 2) mean,
           round(stddev(perf_score("Performance Score")) :: numeric, 2) std, 
           min(perf_score("Performance Score")) min,
           max(perf_score("Performance Score")) max,
           count("Performance Score")
    from racesv
    group by race
    order by mean desc;
    """
)

,race,mean,std,min,max,count
0,Asian,0.76,0.70,-2,2,34
1,White,0.75,0.69,-2,2,193
2,Black or African American,0.68,0.83,-2,2,57
3,Two or more races,0.61,0.85,-2,1,18


Выходит, что представители азиатской рассы и белого наеления в среднем показывают лучшую производительность, чем представители других расс; и вариативность в производительности у первых также меньше (что хорошо).

Помсмотрим, кака связь (и есть ли вообще) между родом деятельности и производительностю представителей разных расс.

In [434]:
sql(
    """
    select racedesc race, 
           department dep, 
           round(avg(perf_score("Performance Score")) :: numeric, 2) mean,
           round(stddev(perf_score("Performance Score")) :: numeric, 2) std, 
           min(perf_score("Performance Score")) min,
           max(perf_score("Performance Score")) max,
           count("Performance Score")
    from racesv
    group by race, dep
    having count("Performance Score") >= 10
    order by dep, mean desc;
    """
)

,race,dep,mean,std,min,max,count
0,Black or African American,IT/IS,1.00,0.67,0,2,10
1,White,IT/IS,0.87,0.43,0,2,30
2,White,Production,0.72,0.70,-2,2,138
3,Asian,Production,0.68,0.78,-2,1,22
4,Two or more races,Production,0.60,0.97,-2,1,10
5,Black or African American,Production,0.56,0.91,-2,2,32
6,White,Sales,0.57,1.09,-2,1,14


К сожалению данных маловато, чтобы можно было проверить гипотизу выше. Из-за большого перекоса в сторону представителей белого населения и департамента производства, можно поставить под сомнения и выводы о разнице в производительности расс в целом.

Посмотрим, как распределена оценка производительности по рассам.

In [435]:
sql(
    """
    with 
        dist as (
            select perf_score("Performance Score") score,
                racedesc, 
                cume_dist() over (
                    partition by racedesc 
                    order by perf_score("Performance Score") desc
                )
            from racesv
            order by score, racedesc
        )
    select distinct * 
    from dist
    order by score, cume_dist desc;
    """
)

,score,racedesc,cume_dist
0,-2,Asian,1.000000
1,-2,Black or African American,1.000000
2,-2,Two or more races,1.000000
3,-2,White,1.000000
4,-1,Black or African American,0.982456
5,-1,Asian,0.970588
6,-1,White,0.968912
7,-1,Two or more races,0.944444
8,0,White,0.943005
9,0,Asian,0.941176


В таблице выше отображены вероятности того, что представитель заданной рассы будем иметь вероятность не меньше указанной.

Из табллицы видно, Asian и White почти всегда быдут производительней всех остальных. А вот работника с оценкой Exceptional вероятнее встретить среди Black or African American. Однако с Exceptional тоже встает проблема недостатка данных (всего 9 человек настолько хороши).

#### Еще одно деикатное "исследование" — производительность в разбивке по полу.

In [436]:
sql(
    """
    select sex, 
           round(avg(perf_score("Performance Score")) :: numeric, 2) mean,
           round(stddev(perf_score("Performance Score")) :: numeric, 2) std, 
           min(perf_score("Performance Score")) min,
           max(perf_score("Performance Score")) max,
           count("Performance Score")
    from hr_dataset hd 
    group by sex
    order by mean desc;
    """
)

,sex,mean,std,min,max,count
0,Female,0.74,0.69,-2,2,177
1,Male,0.71,0.78,-2,2,133


Есть ли склонности к определнной деятельности?

In [437]:

sql(
    """
    select department, sex,  
        round(avg(perf_score("Performance Score")) :: numeric, 2) mean,
        round(stddev(perf_score("Performance Score")) :: numeric, 2) std, 
        min(perf_score("Performance Score")) min,
        max(perf_score("Performance Score")) max,
        count("Performance Score")
    from departmentsv 
    group by department, sex
    order by department, mean desc;
    """
)

,department,sex,mean,std,min,max,count
0,Admin Offices,Male,1.00,0.00,1,1,4
1,Admin Offices,Female,0.67,0.52,0,1,6
2,IT/IS,Female,0.95,0.49,0,2,22
3,IT/IS,Male,0.79,0.63,-1,2,28
4,Production,Female,0.70,0.72,-2,2,127
5,Production,Male,0.68,0.80,-2,2,81
6,Sales,Female,0.80,0.77,-2,1,15
7,Sales,Male,0.50,0.97,-2,1,16
8,Software Engineering,Male,1.25,0.50,1,2,4
9,Software Engineering,Female,0.67,0.82,-1,1,6


В очередной раз данных недостаточно для стабильных выводов. Из того, что есть можно сказать, что "девочки" лучше в IT/IS и Production, а "мальчики" значительно сильнее в Software Engineering и Admin Offices.

Мы много узнали о производительности представительей разных расс, теперь исследуем их лояльность к компании. Как много времени они работают в компании?


In [438]:
sql(
    """
    select racedesc, 
           round(avg("Days Employed") :: numeric, 2)mean, 
           round(stddev("Days Employed") :: numeric, 2) std,
           min("Days Employed"),
           max("Days Employed"),
           count(1)
    from racesv
    group by racedesc 
    order by mean desc;
    """
)

,racedesc,mean,std,min,max,count
0,Two or more races,1628.28,791.85,45,3065,18
1,Black or African American,1436.00,854.20,6,3674,57
2,White,1230.43,733.66,2,4339,193
3,Asian,1164.18,736.50,19,2583,34


In [439]:
sql(
    """
    with 
    dist as (
        select racedesc race, 
               days_to_period_name("Days Employed") period_name, 
               days_to_period("Days Employed") period,
               cume_dist() over (
                   partition by racedesc 
                   order by days_to_period("Days Employed") desc
               ) as dist
        from racesv
    )  
    select distinct * from dist
    order by period, dist desc;
    """
)

,race,period_name,period,dist
0,Asian,less then tree months,0,1.000000
1,Black or African American,less then tree months,0,1.000000
2,Two or more races,less then tree months,0,1.000000
3,White,less then tree months,0,1.000000
4,Black or African American,tree months,1,0.982456
5,White,tree months,1,0.948187
6,Asian,tree months,1,0.941176
7,Black or African American,half a year,2,0.964912
8,White,half a year,2,0.922280
9,Asian,half a year,2,0.911765


Несмотря на некоторые нарекания с производительность Black or African American работают в компании больше всех остальных. А Asian и White вероятнее всего уйдут примерно через год с небольгим.

### Посмотрим, стоит ли вообще напрягаться и быть high-performer.
Другими слловами, если ли связль между производительностью сотрудника и компенсацией труда?

In [440]:
sql(
    """
    select department, 
          perf_score("Performance Score") score, 	   
          round(avg("Pay Rate") :: numeric, 2) mean, 
          round(min("Pay Rate") :: numeric, 2) min,
          round(max("Pay Rate") :: numeric, 2) max,
          count("Pay Rate")
    from departmentsv 
    group by score, department 
    order by department, score;
    """
)

,department,score,mean,min,max,count
0,Admin Offices,0,19.03,16.56,21.50,2
1,Admin Offices,1,35.11,20.50,55.00,8
2,IT/IS,-1,63.00,63.00,63.00,1
3,IT/IS,0,46.74,34.00,61.30,9
4,IT/IS,1,44.38,21.00,64.00,36
5,IT/IS,2,52.06,35.50,65.00,4
6,Production,-2,22.07,16.00,29.00,7
7,Production,-1,24.36,15.00,53.00,11
8,Production,0,21.00,15.00,28.00,25
9,Production,1,23.38,14.00,60.00,161


In [441]:
sql(
    """
    select corr(perf_score("Performance Score"), "Pay Rate")
    from hr_dataset hd;
    """
)

,corr
0,0.054685


In [442]:
sql(
    """
    select department, 
           corr(perf_score("Performance Score"), "Pay Rate")
    from departmentsv 
    group by department 
    order by department;
    """
)

,department,corr
0,Admin Offices,0.499532
1,IT/IS,-0.054152
2,Production,0.031741
3,Sales,0.021390
4,Software Engineering,-0.005861


Невеселая история, выглядит так, что компания не спешит поощрять сотрудников за их успехи.

Как обстоят дела с ростом зп со временем?

In [443]:
sql(
    """
    select department dep, corr("Days Employed", "Pay Rate")
    from departmentsv
    where "Days Employed" > 180
    group by dep
    order by dep;
    """
)

,dep,corr
0,Admin Offices,0.004072
1,IT/IS,-0.155838
2,Production,0.086703
3,Sales,-0.199112
4,Software Engineering,-0.178373


Аналогичная история. Более того, в некоторых отделах компания платит новым сотрудникам больше, чем старым. Зп страрые сотрудники не индексируется. Это может мотивировать их искать новое место работы.

### HR департаменту может быть интересно и полезно знать, как отличаются друг от друга источники найма сторудников.

#### Как долго остаются в компании сотрудники, пришедшие из каждого из источников?

In [444]:
sql(
    """
    select "Employee Source" es, 
           round(avg("Days Employed") :: numeric, 2) mean, 
           round(stddev("Days Employed") :: numeric, 2) std,
           min("Days Employed"), 
           max("Days Employed"), 
           count(1)
    from employeesourcesv 
    group by es 
    having count(1) > 1
    order by mean desc;
    """
)

,es,mean,std,min,max,count
0,Billboard,1680.75,1198.48,62,4339,16
1,MBTA ads,1580.35,569.79,440,2386,17
2,Website Banner Ads,1471.31,516.91,508,2148,13
3,Professional Society,1416.60,534.65,446,2617,20
4,Monster.com,1403.00,757.09,72,2686,24
5,Diversity Job Fair,1340.45,878.48,2,3317,29
6,Search Engine - Google Bing Yahoo,1324.56,932.85,26,3674,25
7,Pay Per Click - Google,1287.29,635.10,58,2512,21
8,Employee Referral,1252.16,561.22,6,2787,31
9,On-campus Recruiting,1214.42,466.00,69,1658,12


#### Откуда приходят лучшие сотрудники?

In [445]:
sql(
    """
    select "Employee Source" es, 
           round(avg(perf_score("Performance Score")) :: numeric, 2) mean, 
           round(stddev(perf_score("Performance Score")) :: numeric, 2) std,
           min(perf_score("Performance Score")), 
           max(perf_score("Performance Score")), 
           count(1)
    from employeesourcesv 
    group by es 
    having count(1) > 1
    order by mean desc;
    """
)

,es,mean,std,min,max,count
0,Employee Referral,0.94,0.51,0,2,31
1,Social Networks - Facebook Twitter etc,0.91,0.30,0,1,11
2,On-campus Recruiting,0.83,0.39,0,1,12
3,Glassdoor,0.79,0.58,-1,1,14
4,Newspager/Magazine,0.78,0.43,0,1,18
5,Search Engine - Google Bing Yahoo,0.76,0.72,-2,1,25
6,Vendor Referral,0.73,0.46,0,1,15
7,MBTA ads,0.71,1.05,-2,2,17
8,Word of Mouth,0.69,0.63,-1,1,13
9,Billboard,0.69,0.95,-2,2,16


#### Посмотрим на оба показателя сразу

In [446]:
sql(
    """
    select "Employee Source" es, 
           round(avg(perf_score("Performance Score")) :: numeric, 2) mean_score, 
           round(avg("Days Employed") :: numeric, 2) mean_days, 
           count(1)
    from employeesourcesv 
    group by es 
    having count(1) > 1
    order by mean_score desc;
    """
)


,es,mean_score,mean_days,count
0,Employee Referral,0.94,1252.16,31
1,Social Networks - Facebook Twitter etc,0.91,1082.73,11
2,On-campus Recruiting,0.83,1214.42,12
3,Glassdoor,0.79,1134.79,14
4,Newspager/Magazine,0.78,1193.78,18
5,Search Engine - Google Bing Yahoo,0.76,1324.56,25
6,Vendor Referral,0.73,1022.07,15
7,MBTA ads,0.71,1580.35,17
8,Word of Mouth,0.69,1022.92,13
9,Billboard,0.69,1680.75,16


Выглядит грустно, ибо чем лучше сотрудник, тем меньше в среднем времени в компании он проведет, и, что еще хуже, — справедливо и обратное утверждение: "плохие" сотрудники работают в компании дольше всего.

Надо это проверить.

In [447]:
sql(
    """
    select "Performance Score" score, 
           round(avg("Days Employed") :: numeric, 2) mean,
           round(stddev("Days Employed") :: numeric, 2) std
    from departmentsv
    group by score
    order by mean desc
    """
)

,score,mean,std
0,Exceptional,1701.33,615.03
1,Exceeds,1627.43,873.66
2,PIP,1426.89,702.08
3,Fully Meets,1402.41,746.96
4,Needs Improvement,1365.87,684.52
5,90-day meets,1055.81,626.97
6,N/A- too early to review,552.41,460.56


К счастью, все не так плохо. Однако странно, что оценку 90-day meets имеют люди, которые проработали в компании больше двух лет. А N/A- too early to review — те, кто проработал полтора года. Какой-то косяк или в данных или в работе HR департамента.

In [448]:
sql(
    """
    with 
    dist as (
        select "Performance Score" score,  
               days_to_period_name("Days Employed") period_name,
               days_to_period("Days Employed") period,
               cume_dist() over (
                   partition by "Performance Score" 
                   order by days_to_period("Days Employed") desc
               ) as dist
        from racesv
        where "Performance Score" not in ('N/A- too early to review', '90-day meets')
    )  
    select distinct * from dist
    order by score desc, period;
    """
)

,score,period_name,period,dist
0,PIP,one year,3,1.000000
1,PIP,two years,5,0.888889
2,PIP,more then two years,6,0.333333
3,Needs Improvement,one year,3,1.000000
4,Needs Improvement,a year and a half,4,0.857143
5,Needs Improvement,two years,5,0.785714
6,Needs Improvement,more then two years,6,0.285714
7,Fully Meets,less then tree months,0,1.000000
8,Fully Meets,tree months,1,0.994350
9,Fully Meets,half a year,2,0.977401


### Выводы и Рекомендации

1. Компании стот обратить внимание на производительность команд под руководством определенных менеджеров и либо заменить отстающих менеджеров либо организовать для них курсы управления для повышения квалификации.
1. Отделы продаж и производства нуждаются в оптимизации процессов либо в постановке более реалистичных KPI.
1. Сточки зрения производительности найм представителей расс Asian и White более выгодный, чем представителей других расс; зато представители Black or African American в среднем проработают в компании дольше.
1. Пересмотреть систему компенсаций таким образом, чтобы сотрудники с более высокой производительностью получали вознаграждение за свои труды; а также, чтобы сотрудники были заинтересованы оставаться в компании с течением времени, а не искать для себя новые возможности на стороне.
1. Не все источники кадров одинково полезны, например, стоит подумать на тем, чтобы отказаться от использования Website Banner Ads, так как сотрудники, пришедгие оттуда показывают, плохую производительность.

В датасете мало данных, из-за этого приходится игнорировать некоторые срезы, а также сложно делать разбивку по несмкольким срезам сразу.

Есть странность, связвнная с несоответствием оценки производительности и временем, отработанным сотрудником в компании. Не доллжно быть так, чтобы среднее время работы в компании сотрудников с оценкой 90 days meets было 1000 дней. Возможно дело в интерпретации значений, например, сотрудник, прошедший PIP снова получает оценку 90 days meets. В таком случае стоит добавить новые оценки, явно показывающие такой переход. Аналогичная история с оценкой N/A- too early to review.




## Часть вторая

Перед вами стоит задача – подготовить аналитический ответ для SMM-отдела компании Skillbox. <br> Объектом анализа является  [паблик Skillbox Вконтакте](https://vk.com/skillbox_education). <br> <br> 
Подключитесь к  API VK и выгрузите посты со стены паблика Skillbox за интересующий период (определите самостоятельно и обоснуйте). Проанализируйте влияние различных факторов (например, времени публикации) на вовлеченность пользователей (количество лайков, комментариев, голосов в опросах). Сделайте аналитику по рубрикам (примеры рубрик: дизайн-битва, игра по управлению), которые есть в паблике. Выбрать нужные посты можно с помощью регулярных выражений. Составьте перечень рекомандаций для SMM-отдела по итогам анализа. <br> <br> 

Дополнительные инструкции по работе с API VK расположены [здесь](https://colab.research.google.com/drive/1rRaTay-OSPLAOX8V9UaFvTiAciVtp2s3).

In [ ]:
### YOUR CODE HERE ###
from pathlib import Path

vk_token = Path('access_token.txt').read_text().rstrip()

import requests

def getjson(url):
    response = requests.get(url)
    response = response.json()
    return response

url = 'https://api.vk.com/method/account.getProfileInfo?access_token=' + str(vk_token) + '&v=5.92'
response = getjson(url)

print(response)


In [450]:
### YOUR CODE HERE ###


